# DELE ST1504 CA2
# PART A: GAN

<hr>

**NAME**: Irman Zafyree, Adam Tan, Joachim

**ADMIN NO**: `2300546`, `<ADM NO.>`, `<ADM NO>`

**CLASS**: DAAA/FT/2B/07

<hr>

**Objective:**

Code a GAN model that is able to generate 260 small black-and-white images of the given dataset in 26 distinct classes.

# Initial Set Up

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

dataset_path = os.environ.get("DATASET_PATH")



emnist-letters-train.csv
